In [24]:
import pandas as pd
from sklearn.cross_validation import train_test_split
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
data = pd.read_csv('./data/session.csv',encoding='gb18030')

In [7]:
data.head()

,用户ID,会话开始时间,咨询类型,咨询总结备注,详情
0,31964968,2019-05-23 23:55:11,售前问题（在线专用）->商品咨询 - -,nan,机器人2(蜜芽智能客服) 2019-05-23 23:55:11\n[xh]甄选洗衣液销售火...
1,32860495,2019-05-23 23:54:58,售中问题（在线专用）->物流问题 - -,nan,机器人4(蜜芽智能客服) 2019-05-23 23:54:58\n[xh]甄选洗衣液销售火...
2,24009062,2019-05-23 23:54:03,售中问题（在线专用）->物流问题 - -,nan,机器人3(蜜芽智能客服) 2019-05-23 23:54:03\n[xh]甄选洗衣液销售火...
3,34644663,2019-05-23 23:53:11,售前问题（在线专用）->商品咨询 - -,nan,机器人2(蜜芽智能客服) 2019-05-23 23:53:11\n[xh]甄选洗衣液销售火...
4,38445693,2019-05-23 23:53:02,售中问题（在线专用）->取消订单 - -,nan,机器人1(蜜芽智能客服) 2019-05-23 23:53:02\n[xh]甄选洗衣液销售火...


In [14]:
data[data['用户ID']=='31964968']['详情']

0    机器人2(蜜芽智能客服) 2019-05-23 23:55:11\n[xh]甄选洗衣液销售火...
Name: 详情, dtype: object

In [17]:
data['咨询类型'].unique()

array(['售前问题（在线专用）->商品咨询 - -', '售中问题（在线专用）->物流问题 - -',
       '售中问题（在线专用）->取消订单 - -', '售后问题（在线专用）->无理由退换货 - -',
       '售后问题（在线专用）->少件 - -', '售后问题（在线专用）->技术问题 - -',
       '售前问题（在线专用）->活动咨询浦发银行 - -', '临时问题（在线专用）->临时问题 - -',
       '规则问题（商管专用）->会员账户 - -', '售前问题（在线专用）->活动咨询（主站活动咨询） - -',
       '售前问题（在线专用）->政策咨询 - -', 'plus分销->PLUS会员提现问题 - -',
       'plus分销->PLUS会员政策咨询 - -', '售后问题（在线专用）->质量问题 - -',
       '售中问题（在线专用）->修改订单 - -', '售后问题（在线专用）->页面与实物不符 - -',
       '售后问题（在线专用）->商品使用咨询 - -', '售后问题（在线专用）->催退款 - -',
       '无效咨询（在线专用）->无效咨询 - -', '社群专用->订单问题（售后问题相关） - -',
       '社群专用->小寰基础操作（登录、修改资料等） - -', '入驻问题（商管专用）->入驻咨询 - -',
       '售前问题（在线专用）->活动咨询（plus问题咨询） - -', 'plus分销->PLUS技术问题 - -',
       '售后问题（在线专用）->发错货 - -', '临时问题（在线专用）->解绑手机 - -',
       '售后问题（在线专用）->保价 - -', '售中问题（在线专用）->催发货 - -',
       '售前问题（在线专用）->活动咨询话费充值 - -', 'plus分销->PLUS大礼包相关 - -',
       '售后问题（在线专用）->破损 - -', '售后问题（在线专用）->发票 - -',
       '社群专用->账户及收益（查销售额、升级业绩等） - -', '售后问题（在线专用）->合作问题 - -',
       '社群专用->售前咨询 - -'

In [20]:
class_mapping = {label:idx for idx,label in enumerate(set(data['咨询类型']))}
data['label'] = data['咨询类型'].map(class_mapping)

In [21]:
data['label'].unique()

array([22, 31,  5, 28, 60, 58, 19,  3, 65, 75, 13, 21, 69, 53, 59, 70, 33,
       36, 14, 24, 42, 40, 47, 74, 37, 71, 41, 35, 54, 52, 48, 56, 23,  8,
       26, 73,  9,  4, 20, 49, 62, 66, 27, 25, 39, 67,  2, 43, 44, 17, 50,
       12, 10, 51, 29, 30, 32,  1, 57, 46, 68,  6, 45,  0, 61, 34, 18, 63,
       55, 16, 11, 64,  7, 15, 38, 72])

In [22]:
len(data['label'].unique())

76

In [30]:
data['detail'] = data['详情'].str.replace('\n', ' ')

In [31]:
train, test = train_test_split(data, test_size=0.2, random_state=2019)
test, dev = train_test_split(test, test_size=0.1, random_state=2019)

In [32]:
print(train.shape, test.shape, dev.shape)

(4000, 7) (900, 7) (100, 7)


In [33]:
train[['label', 'detail']].to_csv('data/mytask/train.tsv', index=False, header=False, sep='\t')
test[['label', 'detail']].to_csv('data/mytask/test.tsv', index=False, header=False, sep='\t')
dev[['label', 'detail']].to_csv('data/mytask/dev.tsv', index=False, header=False, sep='\t')